In [1]:
import pandas as pd
import redis
import pyarrow as pa

In [2]:
excel_data = pd.ExcelFile('../data/hcpcs_data.xlsx')

In [3]:
hcpcs_raw_data = pd.read_excel(excel_data,'hcpcs_raw_data',dtype=str)
doc_mapping = pd.read_excel(excel_data,'doc_mapping',dtype=str)
hcpcs_mapping = pd.read_excel(excel_data,'hcpcs_mapping',dtype=str)

In [4]:
column_header_transformation = lambda x : x.replace(' ','_').lower()


In [5]:
hcpcs_raw_data.columns = map(column_header_transformation,hcpcs_raw_data.columns)
doc_mapping.columns = map(column_header_transformation,doc_mapping.columns)
hcpcs_mapping.columns = map(column_header_transformation,hcpcs_mapping.columns)

hcpcs_raw_data.drop(hcpcs_raw_data.columns[hcpcs_raw_data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)


In [6]:
redis_client = redis.Redis(host='127.0.0.1',port=6379,db=0,password=None)

In [7]:
# Useful link
# https://stackoverflow.com/questions/57949871/how-to-set-get-pandas-dataframes-into-redis-using-pyarrow/57986261#57986261


In [8]:
context = pa.default_serialization_context()

/var/folders/v3/xrqtlz9d0nx4xvkhxwzbg31c0000gn/T/ipykernel_37546/3053119282.py:1: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pa.default_serialization_context()


In [9]:
redis_client.set("hcpcs_raw_data", context.serialize(hcpcs_raw_data).to_buffer().to_pybytes())
redis_client.set("doc_mapping", context.serialize(doc_mapping).to_buffer().to_pybytes())
redis_client.set("hcpcs_mapping", context.serialize(hcpcs_mapping).to_buffer().to_pybytes())

/Users/sarvesh/.pyenv/versions/3.10.3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  if await self.run_code(code, result, async_=asy):


True

In [10]:
redis_client.keys()

[b'hcpcs_mapping', b'hcpcs_raw_data', b'doc_mapping', b'hi']

In [11]:
hcpcs_raw_data = context.deserialize(redis_client.get("hcpcs_raw_data"))
doc_mapping = context.deserialize(redis_client.get("doc_mapping"))
hcpcs_mapping = context.deserialize(redis_client.get("hcpcs_mapping"))

/Users/sarvesh/.pyenv/versions/3.10.3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: 'pyarrow.deserialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  if await self.run_code(code, result, async_=asy):


In [12]:
hcpcs_raw_data_join_hcpcs_mapping = hcpcs_raw_data.merge(hcpcs_mapping,on='hcpcs_code')

In [13]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc = hcpcs_raw_data_join_hcpcs_mapping.merge(doc_mapping,on='npi_doc').reset_index()

In [14]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc = hcpcs_raw_data_join_hcpcs_mapping_npi_doc.applymap(lambda s: s.capitalize() if type(s) == str else s)

In [15]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc['phy_name']=hcpcs_raw_data_join_hcpcs_mapping_npi_doc[['pfname','plname']].apply(lambda x : '{} {}'.format(x[0],x[1]), axis=1)




In [16]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc.head()

,index,year,hcpcs_code,npi_doc,asc_flag,asc_location_flag,srvs_asc_phy,alwd_asc_phy,paid_asc_phy,srvs_ip_phy,...,plname,pfname,doc_specialty,doc_city,doc_state,doc_zip,doc_county,npi_org,org_name,phy_name
0,0,2021,93456,1841290178,1,1,NaN,NaN,NaN,NaN,...,Sanderson,J,Cardiovascular disease physician,Kerrville,Tx,78028,Kerr,1922252717,"Methodist physician practices, pllc",J Sanderson
1,1,2020,93461,1841290178,1,1,NaN,NaN,NaN,NaN,...,Sanderson,J,Cardiovascular disease physician,Kerrville,Tx,78028,Kerr,1922252717,"Methodist physician practices, pllc",J Sanderson
2,2,2019,93461,1841290178,1,1,NaN,NaN,NaN,NaN,...,Sanderson,J,Cardiovascular disease physician,Kerrville,Tx,78028,Kerr,1922252717,"Methodist physician practices, pllc",J Sanderson
3,3,2019,93459,1841290178,1,1,NaN,NaN,NaN,NaN,...,Sanderson,J,Cardiovascular disease physician,Kerrville,Tx,78028,Kerr,1922252717,"Methodist physician practices, pllc",J Sanderson
4,4,2021,93459,1841290178,1,1,NaN,NaN,NaN,NaN,...,Sanderson,J,Cardiovascular disease physician,Kerrville,Tx,78028,Kerr,1922252717,"Methodist physician practices, pllc",J Sanderson


In [17]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc[['op_fac_unit_cost','asc_fac_unit_cost','asc_flag','asc_location_flag','srvs_op_fac']] = hcpcs_raw_data_join_hcpcs_mapping_npi_doc[['op_fac_unit_cost','asc_fac_unit_cost','asc_flag','asc_location_flag','srvs_op_fac']] = hcpcs_raw_data_join_hcpcs_mapping_npi_doc[['op_fac_unit_cost','asc_fac_unit_cost','asc_flag','asc_location_flag','srvs_op_fac']] = hcpcs_raw_data_join_hcpcs_mapping_npi_doc[['op_fac_unit_cost','asc_fac_unit_cost','asc_flag','asc_location_flag','srvs_op_fac']].fillna(0.0)


In [18]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc[['op_fac_unit_cost','asc_fac_unit_cost','asc_flag','asc_location_flag','srvs_op_fac']] = hcpcs_raw_data_join_hcpcs_mapping_npi_doc[['op_fac_unit_cost','asc_fac_unit_cost','asc_flag','asc_location_flag','srvs_op_fac']].astype(float)










In [19]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc['total_savings'] =(hcpcs_raw_data_join_hcpcs_mapping_npi_doc['op_fac_unit_cost'] - hcpcs_raw_data_join_hcpcs_mapping_npi_doc['asc_fac_unit_cost'])* hcpcs_raw_data_join_hcpcs_mapping_npi_doc['asc_flag'] * hcpcs_raw_data_join_hcpcs_mapping_npi_doc['asc_location_flag'] * hcpcs_raw_data_join_hcpcs_mapping_npi_doc['srvs_op_fac']



In [20]:
hcpcs_raw_data_join_hcpcs_mapping_npi_doc.columns

Index(['index', 'year', 'hcpcs_code', 'npi_doc', 'asc_flag',
       'asc_location_flag', 'srvs_asc_phy', 'alwd_asc_phy', 'paid_asc_phy',
       'srvs_ip_phy', 'alwd_ip_phy', 'paid_ip_phy', 'srvs_off', 'alwd_off',
       'paid_off', 'srvs_op_phy', 'alwd_op_phy', 'paid_op_phy',
       'op_fac_unit_cost', 'srvs_op_fac', 'alwd_op_fac', 'paid_op_fac',
       'asc_fac_unit_cost', 'srvs_asc_fac', 'alwd_asc_fac', 'paid_asc_fac',
       'hcpcs_desc', 'hcpcs_group', 'hcpcs_class', 'hcpcs_subclass', 'plname',
       'pfname', 'doc_specialty', 'doc_city', 'doc_state', 'doc_zip',
       'doc_county', 'npi_org', 'org_name', 'phy_name', 'total_savings'],
      dtype='object')

In [21]:
result_set = hcpcs_raw_data_join_hcpcs_mapping_npi_doc.query("hcpcs_code in ['93456'] & year in ['2021']").groupby(['year', 'hcpcs_code', 'npi_doc',
'hcpcs_desc', 'hcpcs_group', 'hcpcs_class', 'hcpcs_subclass','doc_specialty', 'doc_city', 'doc_state', 'doc_zip',
'doc_county', 'npi_org', 'org_name', 'phy_name']).agg({'total_savings':'sum','op_fac_unit_cost':'sum',
'asc_fac_unit_cost':'sum','srvs_asc_phy':'sum','srvs_ip_phy':'sum','srvs_off':'sum','srvs_op_phy':'sum'})

In [22]:
result_set.reset_index(inplace=True)

In [25]:
result_set.query("hcpcs_class in ['Cardiovascular']")

,year,hcpcs_code,npi_doc,hcpcs_desc,hcpcs_group,hcpcs_class,hcpcs_subclass,doc_specialty,doc_city,doc_state,...,npi_org,org_name,phy_name,total_savings,op_fac_unit_cost,asc_fac_unit_cost,srvs_asc_phy,srvs_ip_phy,srvs_off,srvs_op_phy
0,2021,93456,1003217522,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Cardiovascular disease physician,Houston,Tx,...,1881026664,Baylor college of medicine,Srikanth Koneru,0.00,2961.52,1437.45,0,0,0,1
1,2021,93456,1003841297,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Interventional cardiology physician,Plano,Tx,...,1417117912,"Heartplace, pa",Vijay Ramanath,3048.14,2961.52,1437.45,0,1,0,3
2,2021,93456,1003874371,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Cardiovascular disease physician,Sugar land,Tx,...,1487746020,Houston cardiovascular assoc inc,Earl Mangin,1524.07,2961.52,1437.45,0,0,0,0
3,2021,93456,1003886409,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Interventional cardiology physician,San antonio,Tx,...,1487965000,"Cardiology clinic of san antonio, pllc",Jean Liu,3048.14,2961.52,1437.45,0,0,0,2
4,2021,93456,1013143122,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Interventional cardiology physician,Houston,Tx,...,1548287907,Alegent creighton clinic,Arun Kanmantha reddy,1524.07,2961.52,1437.45,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,2021,93456,1972595510,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Interventional cardiology physician,Dallas,Tx,...,1689984585,"Cardiology specialists of north texas, pllc",Eric Eichhorn,0.00,2961.52,1437.45,0,0,0,5
334,2021,93456,1982633970,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Cardiovascular disease physician,Uvalde,Tx,...,1982633970,Unk,Jamil Bitar,0.00,2961.52,1437.45,0,0,0,1
335,2021,93456,1982803458,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Interventional cardiology physician,Fort worth,Tx,...,1801939244,"Fort worth heart, p.a.",Arun Padala,4572.21,2961.52,1437.45,0,0,1,1
336,2021,93456,1982830626,R hrt coronary artery angio,Procedure,Cardiovascular,Percutaneous transcatheterization,Interventional cardiology physician,Galveston,Tx,...,1942241146,Utmb faculty group practice,Paul Kumfa,1524.07,2961.52,1437.45,0,2,0,3
